In [2]:
from datasets import load_dataset,concatenate_datasets, Dataset,DatasetDict
from torch.utils.data import DataLoader
from transformers import AutoTokenizer
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence
import torch

from DMLP.models.my_transformers import MODEL_CLASS
from DMLP.models.models import VAE, DDPM, MLPSkipNet, TransformerNet,VAE_DDPM
from DMLP.train.reconstruction import *
from DMLP.utils.ddpm_schedule import ddpm_schedule
from DMLP.utils.random_init import weights_init_random
from DMLP.train.train_function import train_vae_ddpm
import numpy as np

import json

/home/AD/yul080/.local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/AD/yul080/.local/lib/python3.10/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/AD/yul080/.local/lib/python3.10/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [3]:
tokenizer_encoder = AutoTokenizer.from_pretrained("prajjwal1/bert-small")
tokenizer_decoder = AutoTokenizer.from_pretrained("gpt2-xl")

In [23]:
special_tokens_dict = {'pad_token': '<PAD>', 'bos_token': '<BOS>', 'eos_token': '<EOS>', }
num_added_toks = tokenizer_decoder.add_special_tokens(special_tokens_dict)
gpt2_pad_token = tokenizer_decoder.convert_tokens_to_ids([tokenizer_decoder.pad_token])[0]
gpt2_eos_token = tokenizer_decoder.convert_tokens_to_ids([tokenizer_decoder.eos_token])[0]
gpt2_bos_token = tokenizer_decoder.convert_tokens_to_ids([tokenizer_decoder.bos_token])[0]
bert_pad_token = tokenizer_encoder.pad_token_id
gpt2_pad_token = tokenizer_decoder.pad_token_id



In [16]:
with open('/home/AD/yul080/DMLP/Data/socialiqa-train-dev/train_converted.jsonl') as f:
    train = list((json.loads(data) for data in list(f)))

with open('/home/AD/yul080/DMLP/Data/socialiqa-train-dev/dev_converted.jsonl') as f:
    test = list((json.loads(data) for data in list(f)))
# dataset = Dataset.from_dict({"text":train})

In [26]:
def group_texts(examples):
    question = tokenizer_encoder(
        (examples['context'] + ' ' + examples['question']).replace(".","").replace("?","").replace(":","").replace(",",""), return_special_tokens_mask=True, truncation=True, max_length=512)
    answer = tokenizer_decoder(
            examples['answer'].replace(" ''",'').replace("`` ",''), return_special_tokens_mask=True, truncation=True, max_length=tokenizer_decoder.model_max_length
            )
    return {'question':question['input_ids'],'answer':[tokenizer_decoder.convert_tokens_to_ids([tokenizer_decoder.bos_token])[0]]+answer['input_ids']+[tokenizer_decoder.convert_tokens_to_ids([tokenizer_decoder.eos_token])[0]]}

In [27]:
train_dataset = Dataset.from_list(train)
test_dataset = Dataset.from_list(test)

In [28]:
train_dataset = train_dataset.map(group_texts, batched=False, remove_columns=["context"],num_proc=1)
test_dataset = test_dataset.map(group_texts, batched=False, remove_columns=["context"],num_proc=1)

Map: 100%|██████████| 1954/1954 [00:00<00:00, 4337.54 examples/s]


In [29]:
train_eval_dataset = DatasetDict({'train':train_dataset,'test':test_dataset})

In [30]:
class MyCollator(object):
    def __init__(self, encoder_token, decoder_token):
        self.encoder_token = encoder_token
        self.decoder_token = decoder_token
    def __call__(self, batch):
        question_ids_bert = pad_sequence([torch.tensor(f['question'], dtype=torch.long) for f in batch],
                                  batch_first=True, padding_value=self.encoder_token)
        answer_ids_gpt = pad_sequence([torch.tensor(f['answer'], dtype=torch.long) for f in batch],
                                    batch_first=True, padding_value=self.decoder_token)
        try:
            token_lengths = torch.tensor([[len(f['question']), len(f['answer'])] for f in batch],
                                        dtype=torch.long)
        except:
            token_lengths = torch.zeros((len(batch), 1091))
            for i in range(len(batch)):
                token_lengths[i, len(batch[i]['answer'])] = 1
        return (question_ids_bert, answer_ids_gpt, token_lengths)

In [34]:
batch_size = 1
my_collator = MyCollator(bert_pad_token, gpt2_pad_token)
eval_dataloader =  DataLoader(train_eval_dataset['test'], num_workers=0, collate_fn=my_collator,batch_size=batch_size)
train_dataloader = DataLoader(train_eval_dataset['train'], num_workers=0, collate_fn=my_collator, batch_size=batch_size)

In [35]:
for i in eval_dataloader:
    a = i

In [36]:
a

(tensor([[  101, 18961,  1996,  2961,  2766,  1037,  4062,  2058,  2005, 21485,
           2006,  1996,  2346,  2339,  2106, 18961,  2079,  2023,   102]]),
 tensor([[50258,   568,   661,   836,   470,  3708,   284,  3049, 50259]]),
 tensor([[19,  9]]))

In [38]:
tokenizer_encoder.decode(a[0][0])

'[CLS] aubrey the officer pulled a driver over for speeding on the road why did aubrey do this [SEP]'

In [40]:
tokenizer_decoder.decode(a[1][0])

"<BOS>so people don't drive to fast<EOS>"